In [1]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [2]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.12

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12
     |████████████████████████████████| 2.1 MB 4.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=919aad644179be8c223d6e91f4bb801e1b25909e607340e613d93e5398e8e041
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle


In [3]:
!kaggle competitions download -c tensorflow-great-barrier-reef

100% 14.2G/14.2G [04:04<00:00, 34.1MB/s]
100% 14.2G/14.2G [04:04<00:00, 62.3MB/s]


In [ ]:
!unzip /content/tensorflow-great-barrier-reef.zip -d /content/datasets

Archive:  /content/tensorflow-great-barrier-reef.zip
  inflating: /content/datasets/example_sample_submission.csv  
  inflating: /content/datasets/example_test.npy  
  inflating: /content/datasets/greatbarrierreef/__init__.py  
  inflating: /content/datasets/greatbarrierreef/competition.cpython-37m-x86_64-linux-gnu.so  
  inflating: /content/datasets/test.csv  
  inflating: /content/datasets/train.csv  
  inflating: /content/datasets/train_images/video_0/0.jpg  
  inflating: /content/datasets/train_images/video_0/1.jpg  
  inflating: /content/datasets/train_images/video_0/10.jpg  
  inflating: /content/datasets/train_images/video_0/100.jpg  
  inflating: /content/datasets/train_images/video_0/1000.jpg  
  inflating: /content/datasets/train_images/video_0/1001.jpg  
  inflating: /content/datasets/train_images/video_0/1002.jpg  
  inflating: /content/datasets/train_images/video_0/1003.jpg  
  inflating: /content/datasets/train_images/video_0/1004.jpg  
  inflating: /content/datasets/trai

In [ ]:
!rm -rf "/content/tensorflow-great-barrier-reef.zip"

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5
!pip install -r /content/yolov5/requirements.txt  # install

In [ ]:
import random
import pandas as pd
import numpy
import cv2
from IPython.display import display
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("/content/datasets/train.csv", index_col="image_id")
train.head(10)

In [ ]:
img = cv2.imread("/content/datasets/train_images/video_0/978.jpg")
from google.colab.patches import cv2_imshow

start_pt = (825, 224)
end_pt = (875, 259)
color = (0, 0, 255)
thickness = 2
image = cv2.rectangle(img, start_pt, end_pt, color, thickness)
cv2_imshow(image)
print(image.shape)

In [ ]:
!rm -rf "/content/datasets/coral"

In [ ]:
import os
import ast
import shutil

!mkdir -p datasets/coral/labels
!mkdir -p datasets/coral/images

path_to_txt_folder = "/content/datasets/coral/labels"
final_path_to_images = "/content/datasets/coral/images"

def txt_data(path):
  for folder in os.listdir(path):
    path_to_folder = os.path.join(path, folder)
    _, video_id = folder.split("_")

    for image in os.listdir(path_to_folder):
      path_to_image = os.path.join(path_to_folder, image)
      
      image_id, img_ext = image.split(".")
      image_match = video_id + "-" + image_id
      
      img_temp = cv2.imread(path_to_image)
      height, width, channels = img_temp.shape
      
      image_data = train.loc[image_match]
      annotation_list = ast.literal_eval(image_data.annotations)

      if bool(annotation_list) == True:
        path_to_file = os.path.join(path_to_txt_folder, image_match + ".txt")
        file_ = open(path_to_file, "w")
        for i in annotation_list:
          x_min = i['x']
          y_min = i['y']
          box_width = i['width']
          box_height = i['height']
          class_label=str(0)
          x_min = str(round(x_min/width, 10))
          y_min = str(round(y_min/height, 10))
          box_width = str(round(box_width/width, 10))
          box_height = str(round(box_height/height, 10))
          file_.write(class_label+" "+x_min+" "+y_min+" "+box_width+" "+" "+box_height)
        file_.close()
    
        image_new_name = os.path.join(path_to_folder, image_match + "." + img_ext)
        os.rename(path_to_image, image_new_name)
        shutil.move(image_new_name, final_path_to_images)

txt_data("/content/datasets/train_images")

In [ ]:
img = cv2.imread("/content/datasets/coral/images/0-978.jpg")
from google.colab.patches import cv2_imshow

start_pt = (825, 224)
end_pt = (875, 259)
color = (0, 0, 255)
thickness = 2
image = cv2.rectangle(img, start_pt, end_pt, color, thickness)
cv2_imshow(image)
print(image.shape)

In [ ]:
shutil.move("/content/coral.yaml", "/content/yolov5/data")
shutil.copyfile("/content/hyp.yaml", "/content/yolov5/data/hyps/hyp.yaml")

In [ ]:
!pip install wandb
!wandb login

In [ ]:
# Train YOLOv5m on coral_dataset for 3 epochs
!python /content/yolov5/train.py --img 1280 --batch 8 --epochs 25 --data /content/yolov5/data/coral.yaml  --hyp /content/yolov5/data/hyps/hyp.yaml  --weights yolov5m.pt

wandb: Currently logged in as: ravishkumbhare (use `wandb login --relogin` to force relogin)
train: weights=yolov5m.pt, cfg=, data=/content/yolov5/data/coral.yaml, hyp=/content/yolov5/data/hyps/hyp.yaml, epochs=25, batch_size=8, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-244-g9cf80b7 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1

In [ ]:
import torch
ckpt_path = "/content/yolov5/runs/train/exp4/weights/best.pt"

model = torch.hub.load('/content/yolov5', 'custom',
                           path=ckpt_path,
                           source='local',
                       force_reload=True)

model.conf = 0.003  # NMS confidence threshold
model.iou = 0.45

results = model("/content/datasets/coral/images/0-4600.jpg")
results.print()
results.pandas().xyxy[0]

requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /content/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 v6.0-235-ge1a6a0b torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 290 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 
image 1/1: 720x1280 4 starfishs
Speed: 26.8ms pre-process, 63.4ms inference, 2.4ms NMS per image at shape (1, 3, 384, 640)


,xmin,ymin,xmax,ymax,confidence,class,name
0,700.722473,59.153381,1280.000000,594.391296,0.004380,0,starfish
1,0.000000,71.737091,444.080902,588.231689,0.004047,0,starfish
2,843.220581,29.589203,1268.422974,389.229095,0.003816,0,starfish
3,14.575226,88.720932,938.404053,584.229614,0.003267,0,starfish


In [ ]:
rand_set = random.choices(os.listdir("./datasets/coral/labels"), k=10)
for i in rand_set:
    image, _ = i.split(".")
    file_name = os.path.join("./datasets/coral/images", image + ".jpg")
    results = model(file_name)
    results.print()

image 1/1: 720x1280 4 starfishs
Speed: 27.7ms pre-process, 63.0ms inference, 2.2ms NMS per image at shape (1, 3, 384, 640)
image 1/1: 720x1280 2 starfishs
Speed: 34.1ms pre-process, 63.4ms inference, 2.2ms NMS per image at shape (1, 3, 384, 640)
image 1/1: 720x1280 3 starfishs
Speed: 25.1ms pre-process, 63.2ms inference, 4.2ms NMS per image at shape (1, 3, 384, 640)
image 1/1: 720x1280 4 starfishs
Speed: 26.3ms pre-process, 63.1ms inference, 2.4ms NMS per image at shape (1, 3, 384, 640)
image 1/1: 720x1280 2 starfishs
Speed: 27.6ms pre-process, 63.3ms inference, 2.4ms NMS per image at shape (1, 3, 384, 640)
image 1/1: 720x1280 4 starfishs
Speed: 28.5ms pre-process, 62.5ms inference, 2.4ms NMS per image at shape (1, 3, 384, 640)
image 1/1: 720x1280 3 starfishs
Speed: 24.3ms pre-process, 58.9ms inference, 2.3ms NMS per image at shape (1, 3, 384, 640)
image 1/1: 720x1280 3 starfishs
Speed: 30.4ms pre-process, 58.9ms inference, 5.1ms NMS per image at shape (1, 3, 384, 640)
image 1/1: 720x1